# Semantic Similarity Example

## Setup imports and API keys

First, we'll need to set our API keys. If we are in DEBUG mode, we don't need to use real OpenAI or Hegel AI API keys, so for now we'll set them to empty strings.

In [ ]:
import os
os.environ['DEBUG']="1"
os.environ['HEGELAI_API_KEY'] = ""
os.environ['OPENAI_API_KEY'] = ""

Then we'll import the relevant `prompttools` modules to setup our experiment.

In [ ]:
from typing import Dict, Tuple
from prompttools.harness.prompt_template_harness import (
    PromptTemplateExperimentationHarness,
)

## Run experiments

Next, we create our test inputs. For this example, we'll use a prompt template, which uses [jinja](https://jinja.palletsprojects.com/en/3.1.x/) for templating.

In [ ]:
prompt_templates = ["Echo the following input: {{input}}", "Repeat the following input: {{input}}"]
user_inputs = [{"input": "This is a test"}, {"input": "This is not a test"}]

Now we can define an experimentation harness for our inputs and model. We could also pass model arguments if, for example, we wanted to change the model temperature.

In [ ]:
harness = PromptTemplateExperimentationHarness("gpt-3.5-turbo", prompt_templates, user_inputs)

We can then run the experiment to get results.

In [ ]:
harness.prepare()
harness.run()
harness.visualize()

You can use the `pivot` keyword argument to view results by the template and inputs that created them.

In [ ]:
harness.visualize(pivot=True)

## Evaluate the model response

To evaluate the results, we'll define an eval function. Since we are prompting the model to echo our input, we can use semantic distance to check if the model's response is similar to the user input.

In [ ]:
import chromadb


def extract_responses(output) -> str:
    return [choice["text"] for choice in output["choices"]]


# Define an evaluation function that assigns scores to each inference
def check_similarity(input_pair: Tuple[str,Dict[str,str]], results: Dict, metadata: Dict) -> float:
    chroma_client = chromadb.Client()
    collection = chroma_client.create_collection(name="test_collection")
    collection.add(
        documents=[dict(input_pair[1])['input']],
        ids=["id1"]
    )
    query_results = collection.query(
        query_texts=extract_responses(results),
        n_results=1
    )
    chroma_client.delete_collection("test_collection")
    return min(query_results['distances'])[0]
    


Let's test our similarity function.

In [ ]:
check_similarity((prompt_templates[0], user_inputs[0]), {"choices": [{"text": "This is a test"}, {"text": "This is a text"}]}, {})

Finally, we can evaluate and visualize the results.

In [ ]:
harness.evaluate("did_echo", check_similarity, use_input_pairs=True)
harness.visualize()